In [2]:
class UnitMathOps:
    def __init__(self, value, units, _type, T):
        self.T = T
        self.value = 0.00
        self.units = units.replace(_type, '')
        self.type  = _type
        self.convTable = {'μ': 10**-6,
                          'n': 10**-6,
                          'm': 10**-3, 
                          '' : 1,
                          'k': 10** 3,
                          'M': 10** 6 }
        self.setValue(value, units)

    def __add__(self, other):
        self.assertType('add', other.type)
        val = self.value + other.value
        return self.T(self.getValue(val, self.units), self.units)

    def __sub__(self, other):
        self.assertType('sub', other.type)
        val = self.value - other.value
        return self.T(self.getValue(val, self.units), self.units)

    def __mul__(self, other):
        self.assertType('mul', other.type)
        val = self.value * other.value
        return self.T(self.getValue(val, self.units), self.units)

    def __div__(self, other):
        self.assertType('div', other.type)
        val = self.value / other.value
        return self.T(self.getValue(val, self.units), self.units)

    def __floordiv__(self, other):
        self.assertType('floordiv', other.type)
        val = self.value // other.value
        return self.T(self.getValue(val, self.units), self.units)

    def __mod__(self, other):
        self.assertType('mod', other.type)
        val = self.value % other.value
        return self.T(self.getValue(val, self.units), self.units)

    def __pow__(self, other):
        self.assertType('pow', other.type)
        val = self.value ** other.value
        return self.T(self.getValue(val, self.units), self.units)

    def __abs__(self):
        val = abs(self.value)
        return self.T(self.getValue(val, self.units), self.units)

    def __lt__(self, other):
        self.assertType('compare', other.type)
        return self.value < other.value

    def __le__(self, other):
        self.assertType('compare', other.type)
        return self.value <= other.value

    def __eq__(self, other):
        self.assertType('compare', other.type)
        return self.value == other.value

    def __ne__(self, other):
        self.assertType('compare', other.type)
        return self.value != other.value

    def __ge__(self, other):
        self.assertType('compare', other.type)
        return self.value >= other.value

    def __gt__(self, other):
        self.assertType('compare', other.type)
        return self.value > other.value

    def __str__(self):
        return '{} {}{}'.format(self.getValue(self.value, self.units), 
                                self.units, self.type)
    
    def assertType(self, func, t):
        assert self.type == t, "ERROR: Cannot {} type {} and {}"\
                         .format(func, self.type, t)

    def setValue(self, value, units=''):
        self.value = value * self.convTable[units]
        
    def getValue(self, value, units=''):
        return value / self.convTable[units]

In [3]:
import math

class Power(UnitMathOps):
    def __init__(self, watts, units=''):
        P = lambda w, u: Power(w, u)
        UnitMathOps.__init__(self, watts, units , 'W', P) # P

    def volts(self, other):
        assert other.type in ['A', 'Ω'], 'ERROR: Must be type Amps or Ohms'
        if (other.type == 'A'):
            return Voltage(self.value / other.value)
        else:
            return Voltage(math.sqrt(self.value * other.value))
            
    def amps(self, other):
        assert other.type in ['V', 'Ω'], 'ERROR: Must be type Volts or Ohms'
        if (other.type == 'V'):
            return Current(self.value / other.value)
        else:
            return Current(math.sqrt(self.value / other.value))

    def ohms(self, other):
        assert other.type in ['A', 'V'], 'ERROR: Must be type Amps or Volts'
        if (other.type == 'A'):
            return Resistance(self.value / (other.value**2))
        else:
            return Resistance((other.value**2) / self.value)
        
        
class Voltage(UnitMathOps):
    def __init__(self, volts, units=''):
        V = lambda w, u: Voltage(w, u)
        UnitMathOps.__init__(self, volts, units, 'V', V) # V

    def watts(self, other):
        assert other.type in ['A', 'Ω'], 'ERROR: Must be type Amps or Ohms'
        if (other.type == 'A'):
            return Power(self.value * other.value)
        else:
            return Power((self.value**2) / other.value)
            
    def amps(self, other):
        assert other.type in ['W', 'Ω'], 'ERROR: Must be type Watts or Ohms'
        if (other.type == 'W'):
            return Current(other.value / self.value)
        else:
            return Current(math.sqrt(self.value / other.value))

    def ohms(self, other):
        assert other.type in ['A', 'W'], 'ERROR: Must be type Amps, or Watts'
        if (other.type == 'A'):
            return Resistance(self.value / other.value)
        else:
            return Resistance((self.value**2) / other.value)

class Current(UnitMathOps):
    def __init__(self, amps, units=''):
        C = lambda w, u: Current(w, u)
        UnitMathOps.__init__(self, amps, units, 'A', C) # I

    def watts(self, other):
        assert other.type in ['V', 'Ω'], 'ERROR: Must be type Volts or Ohms'
        if (other.type == 'V'):
            return Power(self.value * other.value)
        else:
            return Power((self.value**2) * other.value)
            
    def volts(self, other):
        assert other.type in ['W', 'Ω'], 'ERROR: Must be type Watts or Ohms'
        if (other.type == 'W'):
            return Voltage(other.value / self.value)
        else:
            return Voltage(self.value * other.value)

    def ohms(self, other):
        assert other.type in ['W', 'V'], 'ERROR: Must be type Watts or Volts'
        if (other.type == 'W'):
            return Resistance(ohter.value / (self.value**2))
        else:
            return Resistance(math.sqrt(other.value/self.value))

class Resistance(UnitMathOps):
    def __init__(self, ohms, units=''):
        R = lambda w, u: Resistance(w, u)
        UnitMathOps.__init__(self, ohms, units, 'Ω', R) # R

    def watts(self, other):
        assert other.type in ['A', 'V'], 'ERROR: Must be type Amps or Volts'
        if (other.type == 'A'):
            return Power(self.value * (other.value**2))
        else:
            return Power((other.value**2) / self.value)
            
    def volts(self, other):
        assert other.type in ['A', 'W'], 'ERROR: Must be type Amps or Watts'
        if (other.type == 'A'):
            return Voltage(self.value * other.value)
        else:
            return Voltage(math.sqrt(self.value * other.value))

    def amps(self, other):
        assert other.type in ['W', 'V'], 'ERROR: Must be type Watts or Volts'
        if (other.type == 'W'):
            return Current(math.sqrt(other.value / self.value))
        else:
            return Current(other.value / self.value)



In [14]:
from copy import deepcopy
from collections import deque
from time import time
import random
'''
PoNS Page 33.    ::   Amplitude(mV)  | Duration                | Summation
Receptor potentials   Small (0.1-10) | Brief (5-100ms)         | Graded
Synaptic potentials   Small (0.1-10) | Brief->Long (5ms-20min) | Graded
Action potentials     Large (70-110) | Brief (1-10ms)          | All-or-none
'''

class Neuron:
    def __init__(self, index_h_i_j, learning_rate):
        self.hashAddress(index_h_i_j)
        self.learning_rate = learning_rate
        self.u_rest = Voltage(-65., 'm')    # Constant membrane potential
        self.u = deepcopy(self.u_rest)      # Membrane potential
        self.𝜗 = Voltage(20, 'm')           # Axon Hillock threshold
        self.preSynaptic = dict()           # {Address: Weight}
        self.postSynaptic = dict()          # {Address: Neuron}
        self.historyStack = deque()         # [Address, Time]
        self.hpt = 0                        # Hyperpolerization time
        self.t = 0                          # Time step (milliseconds)
        random.seed(time())

    def dendrite(self, address, t):
        self.t = t                          # Update current time
        self.historyStack(address)          # Record presynaptic potential
        synapticPotial = self.preSynaptic[address] # Get synaptic weight
        self.u += Voltage(synapticPotial, 'm') # Update membrane potential
        self.PSP = self.u + self.u_rest     # Update postsynaptic potential
        self.axonHillock()                  # Move to Axon Hillock function

    def historyStack(self, address):
        l = len(self.historyStack)          # Record length of stack
        dt = self.t - self.historyStack[0][1] # Record time since oldest spike
        while(l > 0 and dt > 20):           # For all older than 20ms
            self.historyStack.popleft()     # Erase from historyStack
            dt = self.t - self.historyStack[0][1] # Record next spike dt
            l -= 1                          # Update loop condition
        self.historyStack.append([address, self.t]) # Append to historyStack
    
    def STDP(self, depolarization):
        l = len(self.historyStack)
        if (depolarization):
            self.hpt[1] = self.t + 20       # Set hyperpolarization for next 20ms
            for address, t in self.historyStack:
                # Depolarize synaptic weights by learning rate
                dt = self.t - t if (self.t - t > 0) else 1
                self.preSynaptic[address] *= 1 + (self.learning_rate/dt)
        else:
            dhpt = self.hpt - self.t            # Difference in hpt
            if (dhpt > 0):                      # If Hyperpolerization active
                # Hyperpolarize synaptic weight by learning rate
                address = self.historyStack[-1][0]
                self.preSynaptic[address] *= 1 - (self.learning_rate*(dt/20))

    def axonHillock(self):
        if (self.PSP >= self.𝜗):            # If PSP > threshold : FIRE!!!
            self.STDP(True)                     # Activate STDP learning
            self.actionPotential()          # Move to Action Potential function
        else:
            self.STDP(False)

    def actionPotential(self):
        for neuron in self.postSynaptic.values: # For each neuron attached to axon
            neuron.dendrite(self.address, self.t+1) # Activate postsynaptic potential

    def hashAddress(self, index_h_i_j):
        self.address  = hex(index_h_i_j[0])[2:]+'.'
        self.address += hex(index_h_i_j[1])[2:]+'.'
        self.address += hex(index_h_i_j[2])[2:]

In [15]:
n1 = Neuron([55,3,17], 0.001)
n2 = deepcopy(n1)
n1.u += n2.u
print(n1.u)
print(n1.address)

-130.0 mV
37.3.11


"Ó|XG2'¾\x0b¢\x9e±\x8cp|\x9cè"